# 🌪️ TurboSHAKE Hash Implementation

A pure Python implementation of the **TurboSHAKE** cryptographic hash functions - the high-speed variant of the SHA-3 family optimized for performance-critical applications! ⚡

## 🔬 About TurboSHAKE

TurboSHAKE is a revolutionary cryptographic hash function that builds upon the proven SHA-3/Keccak foundation while delivering significantly improved performance. It's designed to be the go-to choice for applications requiring both security and speed.

### 🚀 Key Features

- **⚡ High Performance**: Optimized with reduced rounds (12 instead of 24) for faster computation
- **🔒 Cryptographically Secure**: Built on the battle-tested Keccak-p permutation
- **🎯 Variable Output**: Extendable Output Function (XOF) - generate any length hash
- **🔄 Two Variants**: TurboSHAKE128 and TurboSHAKE256 for different security levels
- **🛡️ Domain Separation**: Built-in domain separation parameter (D = 0x1F)
- **💯 Verified**: 100% test compatibility with reference implementations

## 📚 Technical Specifications

| Algorithm | Rate (bytes) | Capacity (bytes) | Security Level | Rounds |
|-----------|--------------|------------------|----------------|---------|
| TurboSHAKE128 | 168 | 32 | 128-bit | 12 |
| TurboSHAKE256 | 136 | 64 | 256-bit | 12 |

### 🏗️ Architecture

- **Base**: Keccak-p[1600, 12] permutation
- **Structure**: Sponge construction with custom padding
- **Domain Separation**: Uses 0x1F byte for algorithm identification
- **Padding**: Custom padding scheme with 0x80 termination

## 🛠️ Installation & Dependencies

### Prerequisites
- Python 3.7+ 🐍
- Custom SHA-3 implementation (from `HMAC.SHA_3`)
- PyCryptodome (for testing only)

### Setup
```bash
pip install pycryptodome  # For testing against reference implementation
```

## 🧪 Testing & Validation

Our implementation includes comprehensive testing against the official reference implementation:

### Test Results ✅
```
--- TurboSHAKE128 Summary ---
Total tests: 500
Matches   : 500 ✅
Mismatches: 0

--- TurboSHAKE256 Summary ---
Total tests: 500  
Matches   : 500 ✅
Mismatches: 0
```
## 🏗️ Implementation Architecture

### Core Components

```python
# 🔄 Core permutation function
def KECCAK_p_1600_12(S):
    """Keccak-p[1600,12] - 12 rounds instead of 24"""
    
# 🌪️ Main TurboSHAKE function
def turboshake(message, rate_bytes, D=0x1F, outlen=32):
    """Universal TurboSHAKE implementation"""

# 🎯 Convenience wrappers
def turboshake128(message, D=0x1F, outlen=32)
def turboshake256(message, D=0x1F, outlen=32)
```

### Algorithm Flow

1. **📥 Input Processing**: Message + Domain separator (D)
2. **🔄 Padding**: Custom padding to rate boundary
3. **🔚 Termination**: XOR with 0x80 on last byte  
4. **🧽 Sponge Phase**: Absorption using 12-round Keccak-p
5. **💧 Squeeze Phase**: Output generation of desired length

## 🚀 Performance Benefits

### Speed Improvements
- **⚡ 50% faster** than SHA-3 due to reduced rounds
- **🔥 Optimal** for high-throughput applications
- **⚖️ Balanced** security vs. performance trade-off

### Use Cases
- 🌐 **Blockchain applications**: Fast merkle tree construction
- 🔐 **Cryptographic protocols**: Key derivation and random number generation
- 📊 **Data integrity**: High-speed file verification
- 🎮 **Gaming**: Deterministic random generation
- ☁️ **Cloud computing**: Distributed hash computations

## 🔒 Security Analysis

### Cryptographic Properties
- **🛡️ Collision Resistance**: Inherited from Keccak design
- **🎯 Preimage Resistance**: Strong one-way function properties  
- **🔀 Avalanche Effect**: Small input changes cause large output changes
- **📏 Variable Output**: XOF property prevents length extension attacks

### Security Considerations
- ⚠️ **Reduced rounds**: 12 vs 24 rounds - performance vs security trade-off
- ✅ **Proven base**: Built on cryptographically analyzed Keccak
- 🎯 **Domain separation**: Prevents cross-algorithm attacks
- 📋 **Standardization**: Following IACR specifications

## 📖 Academic References

### Primary Reference
**"TurboSHAKE"** - IACR ePrint Archive 2023/342  
🔗 **Paper**: https://eprint.iacr.org/2023/342.pdf  
👥 **Authors**: Guido Bertoni, Joan Daemen, Seth Hoffert, Michaël Peeters, Gilles Van Assche, Ronny Van Keer

### Related Work
- **SHA-3 Standard**: NIST FIPS 202
- **Keccak**: Original sponge construction paper
- **Performance Analysis**: Various implementation studies

## 🔍 Technical Details

### Padding Scheme
```
Message → M || D → Pad to rate → XOR 0x80 to last byte
```

### Rate and Capacity
- **TurboSHAKE128**: r=1344 bits, c=256 bits
- **TurboSHAKE256**: r=1088 bits, c=512 bits

### Domain Separation
- **Default D**: 0x1F (00011111₂)
- **Purpose**: Distinguish from other Keccak-based functions
- **Flexibility**: Customizable for specific applications

---

In [ ]:
#imports from the SHA-3 file . These all are my custom SHA-3 functions 
import sys
import os
sys.path.append('../HMAC')  
from SHA_3 import state_array_to_string, Rnd, string_to_state_array, str_to_keccak_bits, sponge, bin_to_hex_keccak

In [ ]:
def KECCAK_p_1600_12(S):
    A = string_to_state_array(S)
    for ir in range(12, 24):    
        A = Rnd(A, ir)
    return state_array_to_string(A)

def turboshake(message: bytes, rate_bytes: int, D: int = 0x1F, outlen: int = 32) -> bytes:
    r = rate_bytes
    c = 200 - r                  # bytes (200 * 8 = 1600 bits)
    #  M' = M || D
    Mprime = message + bytes([D])
    # pad M' with zeros to multiple of r (if already multiple, keep as it is)
    if len(Mprime) % r != 0:
        Mprime_padded = Mprime + b'\x00' * (r - (len(Mprime) % r))
    else:
        Mprime_padded = Mprime
    #  XOR 0x80 to last byte
    Mprime_padded = bytearray(Mprime_padded)
    Mprime_padded[-1] ^= 0x80
    #  convert to Keccak bit-string and sponge with Keccak-p[1600,12]
    Pbits = str_to_keccak_bits(bytes(Mprime_padded))
    out_bits = sponge(KECCAK_p_1600_12, Pbits, r * 8, outlen * 8)
    return bytes.fromhex(bin_to_hex_keccak(out_bits))

In [23]:
def turboshake128(message: bytes, D: int = 0x1F, outlen: int = 32) -> bytes:
    return turboshake(message, 168, D, outlen)

def turboshake256(message: bytes, D: int = 0x1F, outlen: int = 32) -> bytes:
    return turboshake(message, 136, D, outlen)

In [27]:
import random
import string
from Crypto.Hash import TurboSHAKE128 , TurboSHAKE256


def random_word(min_len=3, max_len=12):
    length = random.randint(min_len, max_len)
    return ''.join(random.choice(string.ascii_letters) for _ in range(length)).encode()

def test_turboshake128(num_tests=500, outlen=32):
    ok_count = 0
    for i in range(num_tests):
        word = random_word()
        ref_xof = TurboSHAKE128.new()
        ref_xof.update(word)
        ref_digest = ref_xof.read(outlen)
        our_digest = turboshake128(word, outlen=outlen)

        match = (our_digest == ref_digest)
        # print("Match" if match else "Mismatch")
        if match:
            ok_count += 1
        else:
            print(f"[Mismatch] word={word!r}")
            print(f"  ref: {ref_digest.hex()}")
            print(f"  ours: {our_digest.hex()}")
            print()
    
    print(f"\n--- Summary ---")
    print(f"Total tests: {num_tests}")
    print(f"Matches   : {ok_count}")
    print(f"Mismatches: {num_tests - ok_count}")
def test_turboshake256(num_tests=500, outlen=64):
    ok_count = 0
    for i in range(num_tests):
        word = random_word()
        ref_xof = TurboSHAKE256.new()
        ref_xof.update(word)
        ref_digest = ref_xof.read(outlen)
        our_digest = turboshake256(word, outlen=outlen)

        match = (our_digest == ref_digest)
        # print("Match" if match else "Mismatch")
        if match:
            ok_count += 1
        else:
            print(f"[Mismatch] word={word!r}")
            print(f"  ref: {ref_digest.hex()}")
            print(f"  ours: {our_digest.hex()}")
            print()
    
    print(f"\n--- Summary ---")
    print(f"Total tests: {num_tests}")
    print(f"Matches   : {ok_count}")
    print(f"Mismatches: {num_tests - ok_count}")    

if __name__ == "__main__":
    random.seed(42)  
    test_turboshake128()
    test_turboshake256()


--- Summary ---
Total tests: 500
Matches   : 500
Mismatches: 0

--- Summary ---
Total tests: 500
Matches   : 500
Mismatches: 0
